In [2]:
import re
import pandas as pd
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook

In [20]:
os.getcwd()

'C:\\Users\\Asus\\Desktop\\Project\\ImageRecommender'

In [8]:
# Counter for unique image names
image_counter = 0

# creating the generator
def reader(filepath, rename=False):
    '''
    This function generates through all the images file in the folder
    ''' 
    current_path = filepath 
    print(current_path)
    
    global image_counter  
    
    for foldername in os.listdir(current_path):
        folderpath = os.path.join(filepath, foldername)
        current_path = folderpath
        
        if os.path.isdir(folderpath) and not foldername.startswith('.'):
            os.chdir(folderpath)
            print(folderpath)
            
            for file in os.listdir(folderpath):
                if file.endswith(".jpg") or file.endswith(".png"): 
                    with open(os.path.join(folderpath, file), "rb") as f:
                        image = f.read()
                        img = cv2.imread(file)
                    
                    # check if image can be loaded correctly
                    if img is not None:   
                        #create new name
                        unique_id = f"{image_counter:07}_"
                        
                        image_counter += 1
                        
                        # yield tuple, so later we need to use
                        yield (unique_id, img, os.path.join(folderpath, file))
                        
                    else:
                        # check path or image format
                        print(f"Error while loading image: {file}")
            
            os.chdir('..')

In [13]:
def get_colour_hist(image):
    
    red_channel = image[:, :, 0]
    green_channel = image[:, :, 1]
    blue_channel = image[:, :, 2]

    # Calculate colour histogram
    red_hist = cv2.calcHist([red_channel], [0], None, [30], [0, 256])
    green_hist = cv2.calcHist([green_channel], [0], None, [30], [0, 256])
    blue_hist = cv2.calcHist([blue_channel], [0], None, [30], [0, 256])

    # Normalize the colour histogram value
    red_hist = cv2.normalize(red_hist, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    green_hist = cv2.normalize(green_hist, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    blue_hist = cv2.normalize(blue_hist, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    return red_hist, green_hist, blue_hist

In [21]:
filepath = os.path.join(os.getcwd(), "weather_image_recognition")
gen = reader(filepath)

In [22]:
# change to `filepath` for creating the csv there
os.chdir(filepath)

In [23]:
# Counter for unique image names
image_counter = 0
gen = reader(filepath) # get histogram values

master_df = pd.DataFrame()
# LOOP OVER ALL IMAGES

for index, image, file_path in tqdm_notebook(gen, desc="extracting RGB information from the images"): # tqdm for terminal 
    #file_name = os.path.basename(file_path).replace(".jpg", "").replace(".png", "") # extract the filename
    rgb_array = get_colour_hist(image)
    
    # extract and format the color values
    red_channel = rgb_array[0]
    green_channel = rgb_array[1]
    blue_channel = rgb_array[2]
   
    # merged the list from RGB channel into one list
    merged_list = np.concatenate((red_channel, green_channel, blue_channel), axis=1).flatten().tolist()
    
    # create a DataFrame, tranpose it, add id and merge to master_df
    df = pd.DataFrame(merged_list)
    df = df.transpose()
    df["image_id"] = index
    df["file_path"] = file_path
    df = df.set_index('image_id')
    
    master_df = pd.concat([master_df, df])

# split the path from the color historgram
path_df = master_df[["file_path"]]
color_df = master_df.drop("file_path", axis="columns")

# save to csv
path_df.to_csv('file_path.csv')
color_df.to_csv('color_histogram.csv')

extracting RGB information from the images: 0it [00:00, ?it/s]

C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition
C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\dew
C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\fogsmog
Error while loading image: 4514.jpg
C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\frost
C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\glaze
C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\hail
C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\lightning
C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\rain
C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\rainbow
C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\rime
C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\sandstorm
C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\snow
Error while load